In [1]:
import spacy
import logging
import pandas as pd
from olaf import Pipeline
from typing import Set, List
from olaf.commons.logging_config import logger
from olaf.data_container import CandidateTerm, Relation, Concept
from olaf.data_container.knowledge_representation_schema import KnowledgeRepresentation
from olaf.pipeline.pipeline_component.term_extraction import (
    POSTermExtraction,
    TFIDFTermExtraction,
    ManualCandidateTermExtraction
    )
from olaf.pipeline.pipeline_component.concept_relation_extraction import (
    CTsToConceptExtraction,
    CTsToRelationExtraction,
    SynonymRelationExtraction,
    SynonymConceptExtraction,
    AgglomerativeClusteringRelationExtraction,
    AgglomerativeClusteringConceptExtraction
)
from olaf.commons.spacy_processing_tools import is_not_punct, is_not_stopword, select_on_pos

from olaf.pipeline.pipeline_component.candidate_term_enrichment import SemanticBasedEnrichment

from olaf.repository.corpus_loader.text_corpus_loader import TextCorpusLoader

/home/oumar/Bureau/ontology-learning/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import numpy as np
import concurrent.futures
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib_venn import venn2, venn3


In [3]:
nlp = spacy.load("en_core_web_lg")

In [4]:
logging.getLogger().setLevel(logging.ERROR)

# Select Corpus

In [5]:
corpus_path = "GC10-DET_doc.txt"
corpus_loader = TextCorpusLoader(corpus_path)

# Select and create relevent concepts from the corpus.


In [6]:
expected_concepts = []
with open("concepts.txt", 'r') as f:
    lines = f.readlines()
    expected_concepts = [concept.rstrip("\n") for concept in lines]
    expected_concepts = [Concept(concept) for concept in expected_concepts]
    f.close()

print(expected_concepts)


[Punching, Welding line, Crescent Gap, Water spot, Oil spot, Silk spot, Inclusion, Rolled pit, Crease, Waist folding, metal surface defect, mechanical failure, drying, mechanical lubricant, temperature, pressure, work roll damage, tension roll damage, local yield, low-carbon]


# Testing concept ratio function

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util


def is_similar(concept_a : str, concept_b: str, nlp =nlp, threshold=.8):
    vector_a, vector_b = nlp(concept_a).vector, nlp(concept_b).vector
    return cosine_similarity([vector_a], [vector_b]) > threshold

def is_equal(concept_a : str, concept_b: str):
    return concept_a.lower() == concept_b.lower()

def hg_lm_similaritiry(concept_a : str, concept_b: str, model=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"), threshold=.8):
    embedding_a, embedding_b = model.encode(concept_a), model.encode(concept_b)
    return util.pytorch_cos_sim(embedding_a, embedding_b) > threshold

def get_concept_ratio(pipeline : Pipeline, expected_concepts : List[Concept], comparator = hg_lm_similaritiry, comparator_args:dict={}) -> tuple:
    """
    Calculate the ratio of expected and unexpected concepts in a given pipeline.

    Parameters
    ----------
    pipeline : Pipeline
        The pipeline object containing concepts.
    expected_concepts : List[Concept]
        A list of expected concepts.kwargs

    Returns
    -------
    Tuple[float, float]: A tuple containing:
        The percentage of expected concepts found in the pipeline.
        The percentage of unexpected concepts in the pipeline.
    """
    
    
    found_concepts = pipeline.kr.concepts
    if len(found_concepts := pipeline.kr.concepts) == 0:
        pipeline.run()
        if len(found_concepts := pipeline.kr.concepts) == 0:
            return (0, 0, 0)

    found_concepts = [found_concept.label for found_concept in found_concepts]
    expected_concepts = [expected_concept.label for expected_concept in expected_concepts]
    expected_concept_occ = 0
    for expected_concept in expected_concepts:
        for found_concept in found_concepts:
            if comparator(expected_concept, found_concept, **comparator_args):
                expected_concept_occ += 1
                break 

    precision = expected_concept_occ/len(expected_concepts)
    recall = expected_concept_occ/len(found_concepts)
    f1 = 2*(precision * recall)/(precision+recall)
    return (precision, recall, f1)

In [8]:
from olaf.pipeline.pipeline_component.term_extraction.manual_candidate_terms import (
    ManualCandidateTermExtraction,
)
from olaf.pipeline.pipeline_component.concept_relation_extraction import (
    CTsToConceptExtraction,
    CTsToRelationExtraction
)

# concept extraction component
concepts = [
    "defect type",
    "steel strip surface",
    "punching",
    "mechanical failure",
    "welding line",
    "coil",
    "weld line",
    "crescent gap",
    "cutting",
    "water spot",
    "drying",
    "oil spot",
    "mechanical lubricant",
    "silk spot",
    "plaque",
    "strip surface",
    "roller",
    "pressure",
    "inclusion",
    "metal surface",
    "spots",
    "fish scale shape",
    "block irregular distribution",
    "rolled pit",
    "bulges",
    "pits",
    "steel plate",
    "work roll",
    "tension roll",
    "damage",
    "crease",
    "fold",
    "uncoiling process",
    "waist folding",
    "deformation",
    "low-carbon"
]

relations = [
    "described",
    "explaining",
    "appears",
    "leads",
    "resulting",
    "changed",
    "produced",
    "drying",
    "caused",
    "affect",
    "appearing",
    "lies",
    "distributed",
    "accompanied",
    "showing",
    "pressed",
    "occurred",
    "circumvented",
    "detected",
    "tracked",
    "results",
    "like",
    "mainly",
    "uncoiling"
]

ct_concept_label = { concept : {concept} for concept in concepts}

manuel_concept_extraction = ManualCandidateTermExtraction(
    ct_label_strings_map=ct_concept_label
)

concept_extraction = CTsToConceptExtraction(
)
# concept extraction component



relation_extraction = CTsToRelationExtraction()
pipelines = []
pipelines.append(
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            manuel_concept_extraction,
            concept_extraction,
        ],
        corpus_loader=corpus_loader
    )
)

In [9]:
current_pipeline = pipelines[-1]
current_pipeline.run()

get_concept_ratio(current_pipeline, expected_concepts)

(0.8, 0.45714285714285713, 0.5818181818181818)

In [10]:
def debug_get_concept_ratio(pipeline : Pipeline, expected_concepts : List[Concept], comparator = hg_lm_similaritiry, comparator_args:dict={}) -> tuple:
    """
    Calculate the ratio of expected and unexpected concepts in a given pipeline.

    Parameters
    ----------
    pipeline : Pipeline
        The pipeline object containing concepts.
    expected_concepts : List[Concept]
        A list of expected concepts.kwargs

    Returns
    -------
    Tuple[float, float]: A tuple containing:
        The percentage of expected concepts found in the pipeline.
        The percentage of unexpected concepts in the pipeline.
    """
    
    found_concepts = pipeline.kr.concepts
    if len(found_concepts := pipeline.kr.concepts) == 0:
        pipeline.run()
        if len(found_concepts := pipeline.kr.concepts) == 0:
            return (0, 0, 0)

    found_concepts = [found_concept.label for found_concept in found_concepts]
    expected_concepts = [expected_concept.label for expected_concept in expected_concepts]
    expected_concept_occ = 0
    for expected_concept in expected_concepts:
        print()
        print(f"{expected_concept} : ", end="")
        for found_concept in found_concepts:
            if comparator(expected_concept, found_concept, **comparator_args):
                print(f"{found_concept} ", end="")
                expected_concept_occ += 1
                break 


    precision = expected_concept_occ/len(expected_concepts)
    recall = expected_concept_occ/len(found_concepts)
    f1 = 2*(precision * recall)/(precision+recall)
    return (precision, recall, f1)



current_pipeline = pipelines[-1]
debug_get_concept_ratio(current_pipeline, expected_concepts)


Punching : punching 
Welding line : weld line 
Crescent Gap : crescent gap 
Water spot : water spot 
Oil spot : oil spot 
Silk spot : silk spot 
Inclusion : inclusion 
Rolled pit : rolled pit 
Crease : crease 
Waist folding : waist folding 
metal surface defect : 
mechanical failure : mechanical failure 
drying : drying 
mechanical lubricant : mechanical lubricant 
temperature : 
pressure : pressure 
work roll damage : 
tension roll damage : tension roll 
local yield : 
low-carbon : low-carbon 

(0.8, 0.45714285714285713, 0.5818181818181818)

## Optimze the similarity threshold


In [11]:
print(get_concept_ratio(current_pipeline, expected_concepts)) # default threshold is 0.8
print(get_concept_ratio(current_pipeline, expected_concepts, comparator_args={"threshold": 0.7}))
print(get_concept_ratio(current_pipeline, expected_concepts, comparator_args={"threshold": 0.6}))
print(get_concept_ratio(current_pipeline, expected_concepts, comparator_args={"threshold": 0.5}))

comparator_args={"threshold": 0.7}

(0.8, 0.45714285714285713, 0.5818181818181818)
(0.9, 0.5142857142857142, 0.6545454545454545)
(0.9, 0.5142857142857142, 0.6545454545454545)
(0.9, 0.5142857142857142, 0.6545454545454545)


# Usefull function

In [12]:
def display_concept(kr: KnowledgeRepresentation) -> None:
    print("Concepts in KR:")
    for concept in kr.concepts:
        print(concept.label)


def display_relation(kr: KnowledgeRepresentation) -> None:
    print("Relations in KR:")
    for relation in kr.relations:
        if (
            relation.source_concept is not None
            or relation.destination_concept is not None
        ):
            print(
                (
                    relation.source_concept.label,
                    relation.label,
                    relation.destination_concept.label,
                )
            )

def describe_pipeline(pipeline: Pipeline) -> None:
    print(pipeline.__class__.__name__)
    for component in pipeline.pipeline_components:
        print(f"\t {component.__class__.__name__}")

# Optimatimizing TFIDF pipeline

In [13]:
from olaf.pipeline.pipeline_component.term_extraction import (
    TFIDFTermExtraction
)

from olaf.pipeline.pipeline_component.concept_relation_extraction import (
    CTsToConceptExtraction,
    SynonymConceptExtraction,
    AgglomerativeClusteringConceptExtraction
)

from joblib import Parallel, delayed

In [14]:
concept_extraction_components = ["CandidatToConcept", "SynonymToConcept", "AgglomerativeClustering"]
candidate_term_treshold = np.arange(0.01, 0.11, 0.01)
results = pd.DataFrame(
    index=concept_extraction_components,
    )

multi_index = pd.MultiIndex.from_product([
   concept_extraction_components, 
    ["Precision", "Rappel", "F1"]
    ])
pipelines_scores = pd.DataFrame(index=candidate_term_treshold, columns=multi_index)

pipelines_scores

CandidatToConcept             SynonymToConcept              \
             Precision Rappel   F1        Precision Rappel   F1   
0.01               NaN    NaN  NaN              NaN    NaN  NaN   
0.02               NaN    NaN  NaN              NaN    NaN  NaN   
0.03               NaN    NaN  NaN              NaN    NaN  NaN   
0.04               NaN    NaN  NaN              NaN    NaN  NaN   
0.05               NaN    NaN  NaN              NaN    NaN  NaN   
0.06               NaN    NaN  NaN              NaN    NaN  NaN   
0.07               NaN    NaN  NaN              NaN    NaN  NaN   
0.08               NaN    NaN  NaN              NaN    NaN  NaN   
0.09               NaN    NaN  NaN              NaN    NaN  NaN   
0.10               NaN    NaN  NaN              NaN    NaN  NaN   

     AgglomerativeClustering              
                   Precision Rappel   F1  
0.01                     NaN    NaN  NaN  
0.02                     NaN    NaN  NaN  
0.03                     NaN    NaN  NaN  
0.04                     NaN    NaN  NaN  
0.05                     NaN    NaN  NaN  
0.06                     NaN    NaN  NaN  
0.07                     NaN    NaN  NaN  
0.08                     NaN    NaN  NaN  
0.09                     NaN    NaN  NaN  
0.10                     NaN    NaN  NaN

In [15]:
tfidf_pipelines = []

tfidf_results = []

In [16]:
## 

def has_not_stopword(terms : Set[CandidateTerm], nlp=nlp,):
    selected_term = []
    for term in terms:
        if is_not_stopword(nlp(term.label)[0]):
            selected_term.append(term)
    return set(selected_term)

def has_not_punct(terms : Set[CandidateTerm], nlp=nlp,):
    selected_term = []
    for term in terms:
        if all([is_not_punct(nlp(char)[0]) for char in term.label]):
            selected_term.append(term)
    return set(selected_term)


bad_pos = ["ADP", "PRON", "CCONJ", "PART", "DET", "INTJ", "SCONJ", "NUM"]

def has_not_bad_pos(terms : Set[CandidateTerm], nlp=nlp, pos_to_avoid: List[str]=bad_pos) -> Set[CandidateTerm]:
    selected_term = []
    for term in terms:
        if not any([doc.pos_ in pos_to_avoid for doc in nlp(term.label)]):
            selected_term.append(term)
    return set(selected_term)

def tidf_postprocessing(terms : Set[CandidateTerm], nlp=nlp) -> Set[CandidateTerm]:

    # has_no_punct = lambda term: any([not is_not_punct(term[i]) for i in range(len(term))])
    terms_doc = [nlp(term.label) for term in terms]
    selected_term = [
        term for term in terms
        if  is_not_punct(nlp(term.label)[0])  
            and is_not_stopword(nlp(term.label)[0])
            and has_not_punct(nlp(term.label)[0])
            and has_not_bad_pos(nlp(term.label)[0])
        ]

    return set(selected_term)

def cts_post_processing(cts: set[CandidateTerm], bad_pos=bad_pos) -> set[CandidateTerm]:
    """Post processing on candidate terms.
    Candidate terms with punctuation, stop words or verbs are removed.

    Parameters
    ----------
    cts: set[CandidateTerm]
        Set of candidate terms to filter.

    Returns
    -------
    set[CandidateTerm]
        The candidate terms validated.    
    """
    existing_cts = []
    new_cts = set()
    for ct in cts:
        keep = True
        if len(ct.corpus_occurrences) > 0:
            for co in ct.corpus_occurrences:
                for token in co:
                    if (not (is_not_punct(token)) or not (is_not_stopword(token)) or (token.pos_ in bad_pos)):
                        keep = False
                        break
        else:
            keep = False
        if keep and ct.label not in existing_cts:
            new_cts.add(ct)
            existing_cts.append(ct.label)
    return new_cts

### TFIDF Term Extraction and Candidat To Concept Extraction

In [17]:
ct_to_concept_pipelines = [
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            TFIDFTermExtraction(
                max_term_token_length=3,
                candidate_term_threshold=threshold,
                cts_post_processing_functions=[has_not_punct, has_not_bad_pos, has_not_stopword]
            ),
            CTsToConceptExtraction()
        ],
        corpus_loader=corpus_loader
    ) for threshold in candidate_term_treshold]


results = []
# Utilisation de ThreadPoolExecutor pour exécuter les pipelines en parallèle
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(get_concept_ratio, pipeline, expected_concepts, hg_lm_similaritiry, comparator_args) 
        for pipeline in ct_to_concept_pipelines
        ]
    

    # Attendre que tous les pipelines soient terminés et collecter les résultats
    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
            results.append(result)
        except Exception as e:
            print(f"Exception occurred: {e}")

for result in results:
    print(result)

/home/oumar/Bureau/ontology-learning/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
print("results :", results)


results : Empty DataFrame
Columns: []
Index: [CandidatToConcept, SynonymToConcept, AgglomerativeClustering]


In [ ]:
ct_to_concept_pipelines1 = [
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            TFIDFTermExtraction(
                max_term_token_length=3,
                candidate_term_threshold=threshold,
                cts_post_processing_functions=[cts_post_processing]
            ),
            CTsToConceptExtraction()
        ],
        corpus_loader=corpus_loader
    ) for threshold in candidate_term_treshold]


results = []
# Utilisation de ThreadPoolExecutor pour exécuter les pipelines en parallèle
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(get_concept_ratio, pipeline, expected_concepts, hg_lm_similaritiry, comparator_args) 
        for pipeline in ct_to_concept_pipelines1
        ]
    

    # Attendre que tous les pipelines soient terminés et collecter les résultats
    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
            results.append(result)
        except Exception as e:
            print(f"Exception occurred: {e}")

for result in results:
    print(result)

Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'
Exception occurred: 'DataFrame' object has no attribute 'append'


: 

### TFIDF Term Extraction and Synonym Concept Extraction

In [ ]:

synonym_concept_pipelines = [
    Pipeline(
        spacy_model=nlp,
        pipeline_components=[
           TFIDFTermExtraction(
                max_term_token_length=3,
                candidate_term_threshold=threshold,
                cts_post_processing_functions=[has_not_punct, has_not_bad_pos, has_not_stopword]
            ),
            SemanticBasedEnrichment(
                threshold=.9
            ),
            SynonymConceptExtraction()
        ],
        corpus_loader=corpus_loader
    ) for threshold in candidate_term_treshold]




results = []
# Utilisation de ThreadPoolExecutor pour exécuter les pipelines en parallèle
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(get_concept_ratio, pipeline, expected_concepts, hg_lm_similaritiry, comparator_args) 
        for pipeline in synonym_concept_pipelines
        ]
    

    # Attendre que tous les pipelines soient terminés et collecter les résultats
    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
            results.append(result)
        except Exception as e:
            print(f"Exception occurred: {e}")

for result in results:
    print(result)

### TFIDF Term Extraction and Agglomerative clustering Concept Extraction

In [ ]:
agglo_clustering_pipelines = [
     Pipeline(
        spacy_model=nlp,
        pipeline_components=[
            TFIDFTermExtraction(
                max_term_token_length=3,
                candidate_term_threshold=threshold,
                cts_post_processing_functions=[has_not_punct, has_not_bad_pos, has_not_stopword]
            ),
            AgglomerativeClusteringConceptExtraction(
                distance_threshold=.3
            )
        ],
        corpus_loader=corpus_loader
    ) for threshold in candidate_term_treshold]

results = []
# Utilisation de ThreadPoolExecutor pour exécuter les pipelines en parallèle
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(get_concept_ratio, pipeline, expected_concepts, hg_lm_similaritiry, comparator_args) 
        for pipeline in agglo_clustering_pipelines
        ]
    

    # Attendre que tous les pipelines soient terminés et collecter les résultats
    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
            results.append(result)
        except Exception as e:
            print(f"Exception occurred: {e}")

for result in results:
    print(result)




### Scores des pipelines

# Visualisation des Resultats

In [ ]:
# Fonction pour créer les diagrammes en barres
def create_bar_chart(index_name, data):
    fig, ax = plt.subplots(figsize=(9, 5))
    bar_width = 0.2
    opacity = 0.8

    # Configurer les positions des barres
    r1 = np.arange(len(data.columns.levels[0]))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]

    precision = data.loc[index_name].xs('Precision', level=1)
    rappel = data.loc[index_name].xs('Rappel', level=1)
    f1 = data.loc[index_name].xs('F1', level=1)

    rects1 = ax.bar(r1, precision, bar_width, alpha=opacity, color='b', label='Précision')
    rects2 = ax.bar(r2, rappel, bar_width, alpha=opacity, color='g', label='Rappel')
    rects3 = ax.bar(r3, f1, bar_width, alpha=opacity, color='r', label='F1')

    ax.set_xlabel('Composants')
    ax.set_ylabel('Scores')
    ax.set_title(f'Scores de Précision, Rappel et F1 pour {index_name}')
    ax.set_xticks([r + bar_width for r in range(len(data.columns.levels[0]))])
    ax.set_xticklabels(data.columns.levels[0])
    ax.legend()

    plt.show()

# Créer un diagramme en barres pour chaque index
# for idx in term_extraction_components:
#     create_bar_chart(idx, pipelines_scores)

In [ ]:
def create_bar_chart(index_name, data):
    precision = data.loc[index_name].xs('Precision', level=1)
    rappel = data.loc[index_name].xs('Rappel', level=1)
    f1 = data.loc[index_name].xs('F1', level=1)

    fig = go.Figure(data=[
        go.Bar(name='Précision', x=data.columns.levels[0], y=precision),
        go.Bar(name='Rappel', x=data.columns.levels[0], y=rappel),
        go.Bar(name='F1', x=data.columns.levels[0], y=f1)
    ])
    
    # Modifier la disposition du graphique
    fig.update_layout(
        title=f'Scores de Précision, Rappel et F1 pour {index_name}',
        xaxis_title='Composants',
        yaxis_title='Scores',
        barmode='group'
    )
    
    fig.show()

data_long = pipelines_scores.reset_index().melt(id_vars='index', var_name=['Composant', 'Métrique'], value_name='Score')
data_long.rename(columns={'index': 'Extraction'}, inplace=True)

def create_bar_chart(index_name, data):
    df = data[data['Extraction'] == index_name]
    fig = px.bar(df, x='Composant', y='Score', color='Métrique', barmode='group',
                 title=f'Scores de Précision, Rappel et F1 pour {index_name}')
    
    fig.update_layout(
        xaxis_title='Composants',
        yaxis_title='Scores'
    )
    fig.update_layout(width=1000, height=600)
    fig.show()

# # Créer un diagramme en barres pour chaque index
# for idx in term_extraction_components:
#     create_bar_chart(idx, data_long)